In [1]:
import pandas as pd
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

In [3]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [5]:
# Merging data based on title
movies = movies.merge(credits, on='title')

In [6]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [7]:
def convert(val):
    try:
        return [i['name'] for i in ast.literal_eval(val)]
    except:
        return []

movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

def get_main_cast(val):
    try:
        return [i['name'] for i in ast.literal_eval(val)[:3]] 
    except:
        return []

movies['cast'] = movies['cast'].apply(get_main_cast)

def get_director(val):
    try:
        for i in ast.literal_eval(val):
            if i['job'] == 'Director':
                return i['name']
    except:
        return ''
    return ''

movies['crew'] = movies['crew'].apply(get_director)


In [8]:
movies['tags'] = movies['overview'] + movies['genres'].astype(str) + movies['keywords'].astype(str) + movies['cast'].astype(str) + movies['crew']

In [9]:
movies.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew,tags
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron,"In the 22nd century, a paraplegic Marine is di..."
1,300000000,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,285,"[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski,"Captain Barbossa, long believed to be dead, ha..."
2,245000000,"[Action, Adventure, Crime]",http://www.sonypictures.com/movies/spectre/,206647,"[spy, based on novel, secret agent, sequel, mi...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes,A cryptic message from Bond’s past sends him o...


In [10]:
movies['tags'].isnull().sum()

3

In [11]:
# Drop null values
movies.dropna(subset=['tags'], inplace=True)

In [12]:
movies['tags'] = movies['tags'].str.lower()

In [13]:
#term frquency 
tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['tags'])

In [14]:
# applying Cosine Similarity
similarity = cosine_similarity(tfidf_matrix)

In [15]:
#to add traliers
from youtubesearchpython import VideosSearch

def get_trailer(movie_name):
    try:
        search = VideosSearch(f"{movie_name} trailer", limit=1)
        return search.result()['result'][0]['link']
    except:
        return "Trailer not found"

>Recommedations Based on Movie

In [16]:
def recommend_by_movie(movie):
    movie = movie.lower()
    if movie not in movies['title'].str.lower().values:
        return "Sorry, movie not found."
    
    index = movies[movies['title'].str.lower() == movie].index[0]
    distances = list(enumerate(similarity[index]))
    
    sorted_movies = sorted(distances, key=lambda x: x[1], reverse=True)[1:11]

    recommendations = []
    for i in sorted_movies:
        title = movies.iloc[i[0]].title
        trailer_link = get_trailer(title)  
        recommendations.append(f"{title} 🎬: {trailer_link}")
    return recommendations

>Recommedations Based on Genre

In [17]:
def recommend_by_genre(genre):
    genre = genre.lower()
    genre_series = movies['genres'].fillna('').astype(str).str.lower()
    genre_movies = movies[genre_series.str.contains(genre)]
    if genre_movies.empty:
        return "Sorry, Try another genre."
    #getting row number of the movie for the given genre
    genre_indices = genre_movies.index.tolist() 
    #finding the avg similarity score for the genre and adding it next to movie index
    mean_similarity = sum(similarity[i] for i in genre_indices) / len(genre_indices)
    distances = list(enumerate(mean_similarity))
    
    sorted_movies = sorted(distances, key=lambda x: x[1], reverse=True)[1:11] 

    recommendations = []
    for i in sorted_movies:
        title = movies.iloc[i[0]].title
        trailer_link = get_trailer(title)  
        recommendations.append(f"{title} 🎬: {trailer_link}")

    return recommendations


In [18]:
recommend_by_movie("Life of pi")

['Deep Rising 🎬: https://www.youtube.com/watch?v=gVFEVXIyu00',
 'The Jungle Book 🎬: https://www.youtube.com/watch?v=T_EN03fJIyY',
 'Sphere 🎬: https://www.youtube.com/watch?v=GJvKLetIV20',
 'Poseidon 🎬: https://www.youtube.com/watch?v=PLcLUEE2pyY',
 "Pirates of the Caribbean: At World's End 🎬: https://www.youtube.com/watch?v=HKSZtp_OGHY",
 'The Blue Lagoon 🎬: https://www.youtube.com/watch?v=A0UapwJIJn8',
 'Titanic 🎬: https://www.youtube.com/watch?v=kVrqfYjkTdQ',
 'Crouching Tiger, Hidden Dragon 🎬: https://www.youtube.com/watch?v=q-HrIQLdaNE',
 'Triangle 🎬: https://www.youtube.com/watch?v=ZEHevCzlDNg',
 'Captain Phillips 🎬: https://www.youtube.com/watch?v=GEyM01dAxp8']

In [19]:
recommend_by_movie("Harry Potter and the chamber of secrets")

['Harry Potter and the Prisoner of Azkaban 🎬: https://www.youtube.com/watch?v=VwErvYgoH70',
 'Harry Potter and the Order of the Phoenix 🎬: https://www.youtube.com/watch?v=LLAaW1EgyY8',
 'Harry Potter and the Goblet of Fire 🎬: https://www.youtube.com/watch?v=80kuiBq95So',
 "Harry Potter and the Philosopher's Stone 🎬: https://www.youtube.com/watch?v=VyHV0BRtdxo",
 'Harry Potter and the Half-Blood Prince 🎬: https://www.youtube.com/watch?v=tAiy66Xrsz4',
 "The Sorcerer's Apprentice 🎬: https://www.youtube.com/watch?v=BOlGRTARxys",
 'Dragonslayer 🎬: https://www.youtube.com/watch?v=vWVJr7JbJZc',
 'Hansel & Gretel: Witch Hunters 🎬: https://www.youtube.com/watch?v=9246msCh7x4',
 'The Craft 🎬: https://www.youtube.com/watch?v=SxEqB--5ToI',
 'Practical Magic 🎬: https://www.youtube.com/watch?v=XmVoYeENOo8']

In [20]:
recommend_by_genre("Fantasy")

['The Chronicles of Narnia: The Voyage of the Dawn Treader 🎬: https://www.youtube.com/watch?v=AON1-Ox-Llc',
 'The NeverEnding Story 🎬: https://www.youtube.com/watch?v=YKGYgFPAP14',
 'Eragon 🎬: https://www.youtube.com/watch?v=MAqm-1zUyno',
 'In the Name of the King: A Dungeon Siege Tale 🎬: https://www.youtube.com/watch?v=Q22A9aRrUIw',
 'Enchanted 🎬: https://www.youtube.com/watch?v=IJwUFq8uOr0',
 "Harry Potter and the Philosopher's Stone 🎬: https://www.youtube.com/watch?v=VyHV0BRtdxo",
 'The Chronicles of Narnia: Prince Caspian 🎬: https://www.youtube.com/watch?v=zZfDS5eu7fA',
 'The Chronicles of Narnia: The Lion, the Witch and the Wardrobe 🎬: https://www.youtube.com/watch?v=usEkWtuNn-w',
 'Pan 🎬: https://www.youtube.com/watch?v=UHl8F60t-ss',
 'Man of Steel 🎬: https://www.youtube.com/watch?v=T6DJcgm3wNY']